In [1]:
# code block for imports 
import pandas as pd 
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
conn = sqlite3.connect('cs506MAPC.db')

## Display first 10 Providers 

In [3]:
qrystr ="SELECT DISTINCT t1.ProviderName,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"


#convert query results into dataframe
df_filtered_providers = pd.read_sql_query(qrystr, conn)
print(df_filtered_providers.head(10))

                     ProviderName     tractNum  MaxAdDown  MaxAdUp      town  \
0  MCI Communications Corporation  25023520100        0.0      0.0  ABINGTON   
1  MCI Communications Corporation  25023520201        0.0      0.0  ABINGTON   
2  MCI Communications Corporation  25023520202        0.0      0.0  ABINGTON   
3          Crown Castle Fiber LLC  25023520100        0.0      0.0  ABINGTON   
4          Crown Castle Fiber LLC  25023520201        0.0      0.0  ABINGTON   
5          Crown Castle Fiber LLC  25023520202        0.0      0.0  ABINGTON   
6                    ViaSat, Inc.  25023520100      100.0      3.0  ABINGTON   
7                    ViaSat, Inc.  25023520201      100.0      3.0  ABINGTON   
8                    ViaSat, Inc.  25023520202      100.0      3.0  ABINGTON   
9          U.S. TelePacific Corp.  25023520202        0.0      0.0  ABINGTON   

   Longitude  Latitude  
0     -70.96     42.12  
1     -70.96     42.12  
2     -70.96     42.12  
3     -70.96     42

## Non Zero Providers

In [7]:
qryStr2 = "SELECT DISTINCT t1.ProviderName,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE MaxAdDown != 0.0 AND MaxAdUp != 0.0 and substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"
df_filtered_non_zero_providers = pd.read_sql_query(qryStr2, conn)
print(df_filtered_non_zero_providers.head(10))


                        ProviderName      town  MaxAdDown  MaxAdUp      town  \
0                       ViaSat, Inc.  ABINGTON      100.0    3.000  ABINGTON   
1  COMCAST CABLE COMMUNICATIONS, LLC  ABINGTON      987.0   35.000  ABINGTON   
2           Verizon New England Inc.  ABINGTON      940.0  880.000  ABINGTON   
3           Verizon New England Inc.  ABINGTON        5.0    0.768  ABINGTON   
4           Verizon New England Inc.  ABINGTON       15.0    1.000  ABINGTON   
5           Verizon New England Inc.  ABINGTON       10.0    1.000  ABINGTON   
6           Verizon New England Inc.  ABINGTON        7.0    0.768  ABINGTON   
7           Verizon New England Inc.  ABINGTON        1.5    0.384  ABINGTON   
8           Verizon New England Inc.  ABINGTON        3.0    0.768  ABINGTON   
9           Verizon New England Inc.  ABINGTON        1.0    0.384  ABINGTON   

   Longitude  Latitude  
0     -70.96     42.12  
1     -70.96     42.12  
2     -70.96     42.12  
3     -70.96     42

## Zero Providers

In [5]:
qryStr3 = "SELECT DISTINCT t1.ProviderName,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE MaxAdDown = 0 AND MaxAdUp = 0 and substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"
df_filtered_zero_providers = pd.read_sql_query(qryStr3, conn)
print(df_filtered_zero_providers.head(10))

                        ProviderName     tractNum  MaxAdDown  MaxAdUp  \
0     MCI Communications Corporation  25023520100        0.0      0.0   
1     MCI Communications Corporation  25023520201        0.0      0.0   
2     MCI Communications Corporation  25023520202        0.0      0.0   
3             Crown Castle Fiber LLC  25023520100        0.0      0.0   
4             Crown Castle Fiber LLC  25023520201        0.0      0.0   
5             Crown Castle Fiber LLC  25023520202        0.0      0.0   
6             U.S. TelePacific Corp.  25023520202        0.0      0.0   
7  Broadview Networks Holdings, Inc.  25023520100        0.0      0.0   
8  COMCAST CABLE COMMUNICATIONS, LLC  25023520201        0.0      0.0   
9  COMCAST CABLE COMMUNICATIONS, LLC  25023520100        0.0      0.0   

       town  Longitude  Latitude  
0  ABINGTON     -70.96     42.12  
1  ABINGTON     -70.96     42.12  
2  ABINGTON     -70.96     42.12  
3  ABINGTON     -70.96     42.12  
4  ABINGTON     -70.9

In [8]:
def generateBaseMap(default_location=[42.360081, -71.058884]):
    base_map = folium.Map(location=default_location)
    return base_map

# download speed
base_map = generateBaseMap()

HeatMap(data=df_filtered_non_zero_providers [['Latitude', 'Longitude', 'MaxAdDown']].groupby(['Latitude', 'Longitude']).
        mean().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map) 

base_map.save('maxaddown.html')

# upload speed
base_map = generateBaseMap()

HeatMap(data=df_filtered_non_zero_providers [['Latitude', 'Longitude', 'MaxAdUp']].groupby(['Latitude', 'Longitude']).
        mean().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map) 

base_map.save('maxadup.html')